# Bioinformatics Platform using JupyterLab

Author: Robert Bradford

This is a thesis project using JupyterLab to facilitate the study of BIOC-4010 course. This tutorial implements the following basic modules using Biopython, NGLView。
1. BLAST
2. Display a 3D structure

Requirements:

To run this notebook successfully, it is recommended to use Miniconda + JupyterLab and install the  required packages and extensions. The notebook shall also work on [Google Colab](https://colab.research.google.com/) or [Binder](https://jupyter.org/binder) but this has not been tested.

The following packages are required and can installed using conda. It is recommended to create a new environment and install these packages. You can use the [nglview-jupyterlab.sh script](https://github.com/nglviewer/nglview/blob/master/devtools/nglview-jupyterlab.sh) to install the nglview related packages.
* python 3.8+
* jupyterlab 2.1+
* biopython 1.7+
* ipywidgets 7.5+
* nodejs 12.0.0+, required for the jupyter-labextensions
* nglview 2.7+
    * if you do not use the nglview-jupyterlab.sh script, run the following two commands manually after you install jupyterlab
    * `jupyter-labextension install @jupyter-widgets/jupyterlab-manager`
    * `jupyter-labextension install nglview-js-widgets@$nglviewversion` where `$nglviewversion` is the version of the `nglview` installed package, which can be inspected with `conda list nglview`.
    

This notebook has been test on:
1. miniconda3
2. 

In [1]:
!pip3 install jupyterlab
!pip3 install biopython
!pip3 install ipywidgets
!pip3 install nglview
!jupyter-labextension install @jupyter-widgets/jupyterlab-manager
!jupyter-labextension install nglview-js-widgets@$nglviewversion

Building jupyterlab assets (production, minimized)
An error occured.
RuntimeError: JupyterLab failed to build
See the log file for details:  /tmp/jupyterlab-debug-ru5vmaim.log
Building jupyterlab assets (production, minimized)
An error occured.
RuntimeError: JupyterLab failed to build
See the log file for details:  /tmp/jupyterlab-debug-4h0pqxuo.log


## Setup environment

In [2]:
import Bio
from Bio.Seq import Seq
import ipywidgets as widgets
from Bio.PDB import *
import os
import sys
import nglview as nv
import platform
print("Python version",sys.version_info)
print("Biopython version", Bio.__version__)

Python version sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
Biopython version 1.78


## Chapter 1 Substitution Matrices
One of the fundamental operations in bioinformatics is comparing two sequences, either nucleic acids or proteins and line them up to archieve maximal levels of identity. This is called _**pairwise sequence alignment**_. Quantification of the similarity of the two sequences is important for establishing whether they are homologs. There are two widely used methods developed to quantify the similarity.

- Method 1: align closely related homologs and counting the frequencies of amino acid substitutions.
- Method 2: use a database of aligned sequences derived from protein domains that have a particular structure or function. The frequencies of amino acid substitutions are recorded.
These two methods gave rise to the *PAM* and *BLOSUM* series of amino acid substitution matrices, respectively. These substitution matrices are used in many sequence alignment tools.

The *Biopython* package includes these substitution matrices.

In [3]:
# this scriptlet display BLOSUM62 and PAM250 matrices
from Bio.Align import substitution_matrices as smatrices
blosum62 = smatrices.load("BLOSUM62")
pam250 = smatrices.load("PAM250")
print(blosum62)
print("-"*80)
print(pam250)

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

## Chapter 2 Molecular Graphics

In this section, we use NGLView to display a structure of interest.


In [4]:
#In this block, the user will be prompted to enter the name of the pdb code that they would like to look at.
text_box = str(input("Please enter the PDB code of the structure: "))

Please enter the PDB code of the structure: 1YY8


In [5]:
# we minic the folder structure of PDB database and save the pdf file in the corresponding folder.
first_pdb_file = PDBList()
name = first_pdb_file.retrieve_pdb_file(text_box)
protein_file = ""
last_value = -1
print(name[-2])

if (platform.system() == 'Windows'):
    while (name[last_value] != '\\'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

else:
    while (name[last_value] != '/'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

protein_file = protein_file[::-1] #The protein name is reversed, to get a proper pdb file format.

protein_name = ""

index = 0
while(protein_file[index] != '.'):
    protein_name += protein_file[index]
    index += 1

protein_name = protein_name.upper()
print("The name of the protein is, " + protein_file)

i
The name of the protein is, 1yy8.cif


In [6]:
protein_class = str(protein_file[1]) + str(protein_file[2])

#We create an instance of the MMCIF Parser, to load the protein file.
parser = MMCIFParser()

path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)
path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)

/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6604.
  warnings.warn(
/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6708.
  warnings.warn(
/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6781.
  warnings.warn(
/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 6879.
  warnings.warn(
/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6604.
  warnings.warn(
/home/faraznaseem/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is d

In [7]:

def clean_protein(obj):
    print(', '.join([a for a in dir(obj) if not a.startswith('_')]))
    
clean_protein(structure)

add, atom_to_internal_coordinates, child_dict, child_list, copy, detach_child, detach_parent, full_id, get_atoms, get_chains, get_full_id, get_id, get_iterator, get_level, get_list, get_models, get_parent, get_residues, has_id, header, id, insert, internal_to_atom_coordinates, level, parent, set_parent, transform, xtra


In [8]:
view_one = nv.show_biopython(structure)
view_one

NGLWidget()

In [9]:
view_one.render_image()

Image(value=b'', width='99%')

In [10]:
view_one._display_image()

In [11]:
view_two = nv.show_biopython(structure)
view_two.add_ball_and_stick()
view_two

NGLWidget()

In [12]:
# clean up the folder if neccessary
os.remove(path) #Removes the file after the user is done looking at the protein.
os.rmdir(protein_class)